## Unit 05. Clustering 課堂作業
- 課程編號: CHEXXXX
- 學年度: 114下
- 上課時間: 每週四 09:00-12:00
-
- 指導教授: ＯＯＯ 教授
- 學生姓名: ＯＯＯ
- 學號: m12345678
- email帳號: fcu.m12345678@gmail.com

---
### 0. 環境設定

In [ ]:
from pathlib import Path
import os

# ========================================
# 路徑設定 (兼容 Colab 與 Local)
# ========================================
UNIT_OUTPUT_DIR = 'P2_Unit05_HW'
SOURCE_DATA_DIR = 'wine_clustering'

try:
  from google.colab import drive
  IN_COLAB = True
  print("✓ 偵測到 Colab 環境，準備掛載 Google Drive...")
  drive.mount('/content/drive', force_remount=True)
except ImportError:
  IN_COLAB = False
  print("✓ 偵測到 Local 環境")
try:
  shortcut_path = '/content/CHE-AI-COURSE'
  os.remove(shortcut_path)
except FileNotFoundError:
  pass

if IN_COLAB:
  source_path = Path('/content/drive/My Drive/Colab Notebooks/CHE-AI-COURSE')
  os.symlink(source_path, shortcut_path)
  shortcut_path = Path(shortcut_path)
  if source_path.exists():
    NOTEBOOK_DIR = shortcut_path / 'Part_2' / 'Unit05'
    OUTPUT_DIR = NOTEBOOK_DIR / 'outputs' / UNIT_OUTPUT_DIR
    DATA_DIR = NOTEBOOK_DIR / 'data' / SOURCE_DATA_DIR
    MODEL_DIR = OUTPUT_DIR / 'models'
    FIG_DIR = OUTPUT_DIR / 'figs'
  else:
    print(f"⚠️ 找不到路徑雲端CHE-AI-COURSE路徑，請確認自己的雲端資料夾是否正確")
  
else:
  NOTEBOOK_DIR = Path.cwd()
  OUTPUT_DIR = NOTEBOOK_DIR / 'outputs' / UNIT_OUTPUT_DIR
  DATA_DIR = NOTEBOOK_DIR / 'data' / SOURCE_DATA_DIR
  MODEL_DIR = OUTPUT_DIR / 'models'
  FIG_DIR = OUTPUT_DIR / 'figs'

NOTEBOOK_DIR.mkdir(parents=True, exist_ok=True)
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
DATA_DIR.mkdir(parents=True, exist_ok=True)
MODEL_DIR.mkdir(parents=True, exist_ok=True)
FIG_DIR.mkdir(parents=True, exist_ok=True)

print(f"\n✓ Notebook工作目錄: {NOTEBOOK_DIR}")
print(f"✓ 數據來源目錄: {DATA_DIR}")
print(f"✓ 結果輸出目錄: {OUTPUT_DIR}")
print(f"✓ 模型輸出目錄: {MODEL_DIR}")
print(f"✓ 圖檔輸出目錄: {FIG_DIR}")

---
### 數據下載

In [ ]:
# 數據來源: Wine Dataset for Clustering
# https://www.kaggle.com/datasets/harrywang/wine-dataset-for-clustering/

import requests
import zipfile

# 1. 設定路徑與 URL
url = "https://www.kaggle.com/api/v1/datasets/download/harrywang/wine-dataset-for-clustering"
zip_path = DATA_DIR / "wine-dataset-for-clustering.zip"
data_file = DATA_DIR / "wine-clustering.csv"
extract_path = DATA_DIR

def download_and_extract():
    # --- 步驟 A: 下載檔案 ---
    print(f"正在從 Kaggle 下載 Wine Dataset for Clustering...")
    print("注意：需要 Kaggle API token 才能下載")
    print("請先在 Kaggle 設定中產生 kaggle.json 並放置在正確位置")
    print("或手動從 https://www.kaggle.com/datasets/harrywang/wine-dataset-for-clustering/ 下載")
    
    try:
        # 使用 kaggle API 下載
        import kaggle
        kaggle.api.dataset_download_files('harrywang/wine-dataset-for-clustering', 
                                         path=DATA_DIR, 
                                         unzip=True)
        print(f"✓ 下載並解壓縮完成")
        
    except Exception as e:
        print(f"⚠️ 自動下載失敗: {e}")
        print(f"請手動下載並解壓縮至: {DATA_DIR}")
        print(f"下載連結: https://www.kaggle.com/datasets/harrywang/wine-dataset-for-clustering/download")

if __name__ == "__main__" and not data_file.exists():
    download_and_extract()
else:
    print(f"✓ 檔案已存在於: {data_file}")

---
### 載入相關套件

In [ ]:
# 基礎套件
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# sklearn 套件 - 資料前處理
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

# sklearn 套件 - 分群模型
from sklearn.cluster import KMeans, AgglomerativeClustering, DBSCAN
from sklearn.mixture import GaussianMixture

# sklearn 套件 - 評估指標
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score
from sklearn.metrics import adjusted_rand_score, normalized_mutual_info_score

# 其他工具
import joblib
from scipy.cluster.hierarchy import dendrogram, linkage
from scipy.spatial.distance import cdist

# 設定隨機種子以確保結果可重現
SEED = 42
np.random.seed(SEED)

# 設定 matplotlib 中文顯示
plt.rcParams['axes.unicode_minus'] = False

print("✓ 套件載入完成")

---
### **I. 練習題: 使用分群方法分析葡萄酒化學成分數據**

本作業使用來自義大利同一地區但不同品種的葡萄酒化學分析數據。數據包含13個化學成分特徵，原始數據有3個品種類別，但在此作業中我們移除類別標籤，進行非監督式分群分析。

**數據集特徵說明**：
- **Alcohol**: 酒精含量
- **Malic_Acid**: 蘋果酸含量
- **Ash**: 灰分
- **Ash_Alcanity**: 灰分的鹼度
- **Magnesium**: 鎂含量
- **Total_Phenols**: 總酚含量
- **Flavanoids**: 類黃酮含量
- **Nonflavanoid_Phenols**: 非類黃酮酚含量
- **Proanthocyanins**: 原花青素含量
- **Color_Intensity**: 顏色強度
- **Hue**: 色調
- **OD280**: OD280/OD315 稀釋葡萄酒的光密度比
- **Proline**: 脯氨酸含量

**資料來源**: [Wine Dataset for Clustering - Kaggle](https://www.kaggle.com/datasets/harrywang/wine-dataset-for-clustering/)

### **1. 數據載入與探索**
- 載入 wine-clustering.csv 數據
- 檢查數據形狀、缺失值、基本統計資訊
- 繪製所有特徵的分布圖 (histogram 或 boxplot)
- 繪製相關係數熱力圖 (correlation heatmap)
- 選擇至少3對特徵繪製散點圖矩陣 (pairplot)

In [ ]:
# 請在此撰寫程式碼


### **2. 數據標準化**
- 對所有特徵進行 StandardScaler 標準化
- 說明為何分群分析需要進行標準化
- 保存標準化後的數據供後續使用

In [ ]:
# 請在此撰寫程式碼


---
## **II. 分群模型建立與訓練**

請使用標準化後的數據，分別建立以下四種分群模型。

### **3. K-Means 分群**

**要求**：
- 使用 Elbow Method 繪製 WCSS (Within-Cluster Sum of Squares) 曲線，測試 k=2 到 k=10
- 使用 Silhouette Score 找出最佳 k 值
- 使用最佳 k 值建立 K-Means 模型
- 繪製分群結果的 2D 視覺化圖 (使用 PCA 降維至 2D)
- 計算並記錄評估指標：Silhouette Score、Davies-Bouldin Index、Calinski-Harabasz Score
- 保存模型至 MODEL_DIR

In [ ]:
# 請在此撰寫程式碼


### **4. Hierarchical Clustering (階層式分群)**

**要求**：
- 使用 linkage='ward' 建立階層式分群模型
- 繪製樹狀圖 (dendrogram) 並根據樹狀圖決定最佳分群數
- 使用最佳分群數建立模型並預測分群標籤
- 繪製分群結果的 2D 視覺化圖 (使用 PCA 降維至 2D)
- 計算並記錄評估指標：Silhouette Score、Davies-Bouldin Index、Calinski-Harabasz Score
- 保存模型至 MODEL_DIR

In [ ]:
# 請在此撰寫程式碼


### **5. DBSCAN 分群**

**要求**：
- 使用網格搜尋 (Grid Search) 測試不同的 eps 和 min_samples 參數組合
- 選擇產生最佳 Silhouette Score 的參數
- 使用最佳參數建立 DBSCAN 模型
- 繪製分群結果的 2D 視覺化圖，並標示出噪音點 (label=-1)
- 計算並記錄評估指標 (排除噪音點)：Silhouette Score、Davies-Bouldin Index、Calinski-Harabasz Score
- 記錄噪音點數量與比例
- 保存模型至 MODEL_DIR

In [ ]:
# 請在此撰寫程式碼


### **6. Gaussian Mixture Model (GMM) 分群**

**要求**：
- 使用 BIC (Bayesian Information Criterion) 和 AIC (Akaike Information Criterion) 選擇最佳分群數 (測試 n_components=2 到 10)
- 使用最佳分群數建立 GMM 模型
- 繪製 BIC 和 AIC 曲線圖
- 繪製分群結果的 2D 視覺化圖 (使用 PCA 降維至 2D)
- 計算並記錄評估指標：Silhouette Score、Davies-Bouldin Index、Calinski-Harabasz Score
- 視覺化每個分群的機率分布 (可選)
- 保存模型至 MODEL_DIR

In [ ]:
# 請在此撰寫程式碼


---
## **III. 模型綜合評估與比較**

### **7. 建立評估指標比較表**

**要求**：
- 整理所有四個模型的評估指標 (Silhouette Score, Davies-Bouldin Index, Calinski-Harabasz Score)
- 使用 pandas DataFrame 建立比較表格
- 繪製評估指標的柱狀圖比較 (每個指標一張圖)
- 分析並說明哪個模型表現最好，為什麼？

In [ ]:
# 請在此撰寫程式碼


### **8. 四個模型的視覺化比較**

**要求**：
- 在同一張圖上繪製四個模型的分群結果 (2x2 子圖)
- 使用 PCA 降維至 2D
- 每個子圖標題顯示模型名稱和分群數
- 使用相同的配色方案
- 保存圖片至 FIG_DIR

In [ ]:
# 請在此撰寫程式碼


---
## **IV. 額外加分作業 (自由繳交)**

- 學生可自由選擇是否繳交加分作業 (下週上課前完成email電子檔)
- 分數會加在最後總成績上，每份作業加 0.1 ~ 1.0 分
- 分數加的不多，真的不一定要交加分作業，正常出席上課做好期末報告即可
- 加分作業不一定要全部都做完，想繳交至少完成其中一項實驗即可
- 請務必自行完成！你可以問AI、問同學、問學長姊，甚至參考以前別人的作業，但一定要自行[吸收][執行][整理]結果！
- 不要貼AI的答案寄給老師看，你自己看就好
- 如果系統自動比對發現作業內容(與他人雷同、原創性低、抄襲比例過高)，作業的分數會0分
- 如果你直接100%複製別人的作業繳交，你會直接被當掉(提供作業給別人複製的也一樣)
- 老師看你作業要花時間，真的有心想寫加分作業，請好好自己做

### **實驗1: 不同標準化方法對分群結果的影響**

比較以下三種資料前處理方法對 K-Means 分群的影響：
- **無標準化**: 直接使用原始數據
- **StandardScaler**: Z-score 標準化
- **MinMaxScaler**: Min-Max 正規化 (範圍 0-1)

**要求**：
- 使用相同的 K-Means 參數 (k=3, random_state=42)
- 記錄每種方法的評估指標 (Silhouette Score, Davies-Bouldin Index, Calinski-Harabasz Score)
- 繪製三種方法的分群結果比較圖 (1x3 子圖)
- **分析**: 哪種標準化方法最適合？為什麼？標準化對分群結果有什麼影響？

In [ ]:
# 請在此撰寫程式碼


### **實驗2: 不同初始化方法對 K-Means 的影響**

比較 K-Means 的不同初始化方法：
- **k-means++**: sklearn 預設方法
- **random**: 隨機初始化

**要求**：
- 使用相同的 k=3, random_state=42
- 對每種初始化方法執行 10 次，記錄每次的 Silhouette Score 和 Inertia
- 計算每種方法的平均值和標準差
- 繪製箱型圖比較兩種方法的穩定性
- **分析**: k-means++ 相比 random 有什麼優勢？為什麼？

In [ ]:
# 請在此撰寫程式碼


### **實驗3: Hierarchical Clustering 不同 Linkage 方法比較**

比較 Hierarchical Clustering 的不同鏈結方法：
- **ward**: 最小化組內變異數
- **complete**: 最大距離鏈結
- **average**: 平均距離鏈結
- **single**: 最小距離鏈結

**要求**：
- 使用相同的分群數量 (n_clusters=3)
- 繪製四種方法的樹狀圖 (dendrogram) 比較
- 記錄每種方法的評估指標 (Silhouette Score, Davies-Bouldin Index, Calinski-Harabasz Score)
- 繪製四種方法的分群結果比較圖 (2x2 子圖)
- **分析**: 不同 linkage 方法對分群結果有什麼影響？哪種方法最適合此數據集？為什麼？

In [ ]:
# 請在此撰寫程式碼


### **實驗4: GMM 不同協方差類型 (Covariance Type) 的影響**

比較 GMM 的不同協方差矩陣類型：
- **full**: 完全協方差矩陣（每個分量有獨立的完整協方差矩陣）
- **tied**: 綁定協方差矩陣（所有分量共用相同的協方差矩陣）
- **diag**: 對角協方差矩陣（僅考慮對角元素）
- **spherical**: 球形協方差矩陣（各維度獨立且方差相同）

**要求**：
- 使用相同的 n_components=3, random_state=42
- 記錄每種類型的 BIC、AIC 值
- 記錄每種類型的評估指標 (Silhouette Score, Davies-Bouldin Index, Calinski-Harabasz Score)
- 繪製四種類型的分群結果比較圖 (2x2 子圖)
- 繪製 BIC 和 AIC 比較柱狀圖
- **分析**: 不同協方差類型對分群結果有什麼影響？哪種類型最適合此數據集？為什麼？

In [ ]:
# 請在此撰寫程式碼


### **實驗5: DBSCAN 距離度量 (Distance Metric) 的影響**

比較 DBSCAN 使用不同的距離度量方法：
- **euclidean**: 歐幾里得距離（預設）
- **manhattan**: 曼哈頓距離
- **cosine**: 餘弦距離
- **chebyshev**: 切比雪夫距離

**要求**：
- 對每種距離度量方法，使用相同的 eps 和 min_samples 參數（可先用 euclidean 找最佳參數）
- 記錄每種方法的分群數量、噪音點數量與比例
- 記錄每種方法的評估指標 (Silhouette Score, Davies-Bouldin Index, Calinski-Harabasz Score)
- 繪製四種方法的分群結果比較圖 (2x2 子圖)
- **分析**: 不同距離度量對 DBSCAN 分群結果有什麼影響？在什麼情況下應該選擇哪種距離度量？

In [ ]:
# 請在此撰寫程式碼


### **實驗6: 降維後分群 vs 原始數據分群的效果比較**

比較不同維度下 K-Means 分群的效果：
- **原始數據**: 使用全部13個特徵
- **PCA降維至5維**: 保留95%以上的變異數
- **PCA降維至3維**: 方便視覺化
- **PCA降維至2維**: 方便視覺化

**要求**：
- 對每種維度，使用相同的 k=3, random_state=42
- 記錄每種維度的評估指標 (Silhouette Score, Davies-Bouldin Index, Calinski-Harabasz Score)
- 記錄每種維度的累積解釋變異數比例
- 繪製四種維度的分群結果比較圖（都投影到2D顯示）
- 繪製評估指標隨維度變化的折線圖
- **分析**: 降維對分群結果有什麼影響？在什麼情況下應該先降維再分群？降維的優缺點是什麼？

In [ ]:
# 請在此撰寫程式碼


---
## **V. 💭 思考題**

請根據本次作業的實作經驗，回答以下問題：

1. **為什麼在分群分析中需要對數據進行標準化？如果不標準化會有什麼問題？**

2. **K-Means、Hierarchical Clustering、DBSCAN、GMM 這四種分群方法各有什麼優缺點？在什麼情況下應該選擇哪一種？**

3. **Silhouette Score、Davies-Bouldin Index、Calinski-Harabasz Score 這三個評估指標分別代表什麼意義？數值越大越好還是越小越好？**

4. **如何決定最佳的分群數量 k？Elbow Method、Silhouette Score、BIC/AIC 各有什麼優缺點？**

5. **DBSCAN 的 eps 和 min_samples 參數如何影響分群結果？如何選擇合適的參數值？**

6. **在葡萄酒化學成分分群中，如果要將結果應用於實際的葡萄酒分類，你會選擇哪個模型？為什麼？**

7. **如果數據中存在噪音點或離群值，哪種分群方法最適合？為什麼？**

8. **GMM 相比 K-Means 有什麼優勢？在什麼情況下 GMM 會比 K-Means 表現更好？**

9. **如何評估分群結果的品質？除了本作業使用的評估指標外，還有哪些方法可以驗證分群的有效性？**

10. **在化工領域中，分群分析可以應用在哪些實際問題上？請舉例說明。**

**請在此撰寫你的回答**


---
# 想對老師說的話